In [1]:
import ezdxf
from ezdxf.addons import MTextExplode
from shapely.geometry import LineString, Polygon, Point
from openpyxl import Workbook
from pprint import pprint
from ezdxf.addons import odafc

In [117]:
# Функция для преобразования MText в текст
# def convert_mtext_to_text(doc):
#     # FIXME: Текст преобразуется с символами форматирования
#     for mtext in doc.modelspace().query('MTEXT'):
#         text = doc.modelspace().add_text(mtext.dxf.text, dxfattribs={
#             'insert': mtext.dxfattribs()["insert"],
#             'height': mtext.dxfattribs()["char_height"],
#             'style': mtext.dxfattribs()["style"],
#         })
#         doc.modelspace().delete_entity(mtext)

# def convert_text_to_mtext(doc):
#         for text in doc.modelspace().query('TEXT'):
#             mtext = doc.modelspace().add_mtext(text.dxf.text, dxfattribs={
#                 'insert': text.dxfattribs()["insert"],
#                 'char_height': text.dxfattribs()["height"],
#                 'style': text.dxfattribs()["style"],
#             })
#             doc.modelspace().delete_entity(mtext)

def explode_mtext(doc):
    # FIXME: Текст с обозначением площади разбивается на две строки
    # FIXME: Возможно удастся исправить через регулярные выражение
    msp = doc.modelspace()
    with MTextExplode(msp) as xpl:
        for mtext in msp.query("MTEXT"):
            xpl.explode(mtext)

In [5]:
# Функция для определения центра текста
def get_text_center(text):
    bbox = text.bbox()
    center_x = (bbox.extmin.x + bbox.extmax.x) / 2
    center_y = (bbox.extmin.y + bbox.extmax.y) / 2
    return center_x, center_y

In [47]:
# Функция для создания бесконечных линий на основе отрезков
def create_infinite_lines(entities):
    infinite_lines = []
    for entity in entities:
        if entity.dxftype() in ['LINE', 'LWPOLYLINE', 'POLYLINE']:
            vec_start = list(entity.vertices_in_wcs())[0]
            vec_end = list(entity.vertices_in_wcs())[-1]
            start = [vec_start.x, vec_start.y]
            end = [vec_end.x, vec_end.y]
            # Создаем бесконечные линии, задавая их как LineString
            line = LineString([start, end]).parallel_offset(1000, 'left').parallel_offset(1000, 'right')
            infinite_lines.append(line)
    return infinite_lines

In [3]:
# Функция для нахождения замкнутых прямоугольников (ячейки таблицы)
def find_rectangles(infinite_lines):    # FIXME: добавить функцию очистки ячеек по площади
    rectangles = []
    for i, line1 in enumerate(infinite_lines):
        for j, line2 in enumerate(infinite_lines):
            if i != j:
                # Проверяем, пересекаются ли линии
                intersection = line1.intersection(line2)
                if isinstance(intersection, Point):
                    # Если пересекаются, создаем прямоугольники
                    rect = Polygon([line1.coords[0], line1.coords[1], line2.coords[0], line2.coords[1]])
                    rectangles.append(rect)
    return rectangles

In [30]:
# Функция для записи данных в Excel
def write_to_excel(data):
    wb = Workbook()
    ws = wb.active
    ws.title = "Table Data"

    # Записываем данные
    for i, row in enumerate(data, start=1):
        for j, value in enumerate(row, start=1):
            ws.cell(row=i, column=j, value=value)

    wb.save("data/ODA/table_data.xlsx")

In [118]:
file_path = 'data/dwg/улица Проектируемая № 56.ODA.dxf'
doc = ezdxf.readfile(file_path)

# Преобразуем MText в Text
# explode_mtext(doc)

# # Определяем координаты центров текстовых объектов
# mtext_entities = doc.modelspace().query('MTEXT')
# text_centers = [(mtext, get_text_center(mtext)) for mtext in mtext_entities]
#
# # Преобразуем все линии в бесконечные отрезки
# lines = doc.modelspace().query('LINE LWPOLYLINE POLYLINE')
# infinite_lines = create_infinite_lines(lines)
#
# # Находим замкнутые прямоугольники (ячейки)
# rectangles = find_rectangles(infinite_lines)
#
# # Связываем текст с ячейками
# data = []
# for mtext, center in text_centers:
#     for rect in rectangles:
#         if rect.contains(Point(center)):
#             data.append([mtext.get_text(), center, rect.bounds])
#             break
#
# # Записываем результат в Excel
# write_to_excel(data)

doc.saveas("data/ODA/test1_output.ODA")

In [35]:
dwg_path = 'data/dwg/ACAD-гск-7.dxf'
doc = ezdxf.readfile(dwg_path)
mt_list = [[mt.plain_text().replace('\n', ' '), mt.dxf.insert[1], mt.dxf.insert[0]] for mt in doc.modelspace().query('MTEXT')]
mt_list_sorted_by_x = sorted(mt_list, key=lambda m: m[1])[::-1]
name_group = mt_list_sorted_by_x.pop(0)[0]
print(name_group)
n_coloumns = 5
data_array = [mt_list_sorted_by_x[i:i + n_coloumns] for i in range(0, len(mt_list_sorted_by_x), n_coloumns)]
for idx, row in enumerate(data_array):
    data_array[idx] = sorted(data_array[idx], key=lambda m: m[2])
table_array = []
for row in data_array:
    table_array.append([mt[0] for mt in row])
pprint(table_array)

улица Тихая 1-я
[['3193', 'Ива ломкая (7 стволов)', '1', '2,2,2,2, 2,2,2', '3,3,3,3, 3,3,3'],
 ['3194', 'Ива ломкая (7 стволов)', '1', '2,2,2,2, 2,2,2', '3,3,3,3, 3,3,3'],
 ['3195', 'Ива ломкая (7 стволов)', '1', '2,2,2,2, 2,2,2', '3,3,3,3, 3,3,3'],
 ['3196', 'Ива ломкая (7 стволов)', '1', '2,2,2,2, 2,2,2', '3,3,3,3, 3,3,3'],
 ['3197', 'Ива ломкая (7 стволов)', '1', '2,2,2,2, 2,2,2', '3,3,3,3, 3,3,3'],
 ['3198', 'Берёза пушистая', '1', '3,5', '5'],
 ['3199', 'Ива ломкая (7 стволов)', '1', '2,5х7', '3,3,3,3, 3,3,3'],
 ['3200', 'Ива ломкая (7 стволов)', '1', '2,5х7', '3,3,3,3, 3,3,3'],
 ['3201', 'Ива ломкая (7 стволов)', '1', '2,5х7', '3,3,3,3, 3,3,3'],
 ['3202', 'Ива ломкая (7 стволов)', '1', '2,2,2,2, 2,2,2', '3,3,3,3, 3,3,3'],
 ['3203', 'Ива ломкая', '1', '1,5', '2'],
 ['3204', 'Смородина черная', '1', 'свыше 1м', '-'],
 ['3205', 'Ива ломкая (6 стволов)', '1', '2,2,2, 2,2,2', '4,4,4, 4,4,4'],
 ['3206', 'Смородина черная', '1', 'свыше 1м', '-']]
